In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19879 sha256=bcd573e1eb1c0924247c8031ff80fbff9ee989294203cf007e7627d1cce147cc
  Stored in directory: /root/.cache/pip/wheels/31/49/9c/44b13823eb256a3b4dff34b972f7a3c7d9910bfef269e59bd7
Successfully built pyngrok


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

from fastapi import FastAPI

import datetime
import pandas as pd
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

ngrok_key = input("Your ngrok Authtoken")

!ngrok authtoken {ngrok_key}

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=tjNZORNvLC1dFEjV-YVxKFOoHYiArziQrBKBeZ1e-9Q&tc=gBtJYo3FhxymSgUFqKjk1hjsJGtmztNXT3gfRU372kE&cc=-FXgF4pccl06B7SvtzyCesmFLkoDCP1798_KjCXkaL0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7RafZkQJYmhY7MLCViRKkSTrVd14D7-F6AwpbuvnOsZTf-SgLSK04

Successfully saved authorization token.
Your ngrok Authtoken2JkgPPlX3ny9g1wWL2II9h5GA9n_3jVqkqfz7ZnBx2Neqbgsv
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
def generate_collection(geometry, dataset, startDate, endDate):
    collection = ee.ImageCollection(dataset).filterDate(startDate, endDate).filterBounds(geometry);
    return collection

def get_mean(image_collection, point, property, scale_factor, sample_scale):
    image = image_collection.select([property]).reduce(ee.Reducer.mean()).multiply(scale_factor)

    fcPoint = ee.FeatureCollection([ee.Feature(point)])
              
    geojson = image.sampleRegions(collection = fcPoint,  scale = sample_scale, geometries = True)

    result = None

    temp_result = geojson.getInfo()

    if temp_result and "features" in temp_result and len(temp_result["features"]) > 0 and "properties" in temp_result["features"][0]:
        result = temp_result["features"][0]["properties"]

    return result

def generate_image(dataset):
    return ee.Image(dataset)

def get_image_value(image, point, property, scale_factor):

    result = None
    temp_result = image.reduceRegion(ee.Reducer.mean(), point, 100).get(property).getInfo()

    if temp_result:
        result = temp_result
    return image.reduceRegion(ee.Reducer.mean(), point, 100).get(property).getInfo() * scale_factor

#point = ee.Geometry.Point([-92.03593750000002, 52.72389418745157])

#image_collection = generate_collection(point, "MODIS/061/MOD11A1", "2000-01-01", "2020-05-01")

#get_mean(image_collection, point, "LST_Day_1km", 0.02)

In [ ]:
app = FastAPI()

@app.get('/')
async def root():
    return {'hello': 'world'}

In [ ]:
@app.get("/land-surface-temperature")
async def get_land_surface_temperature(lat: float, lon: float, start_date: datetime.date, end_date: datetime.date):
    dataset = "MODIS/061/MOD11A1"

    point = ee.Geometry.Point([lon, lat])

    image_collection = generate_collection(point, dataset, start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))

    result = get_mean(image_collection, point, "LST_Day_1km", 0.02, 1000)
    
    return {'result': result}

In [ ]:
@app.get("/soil-ph")
async def get_soil_ph(lat: float, lon: float):

    dataset = "OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02"
    
    point = ee.Geometry.Point([lon, lat])

    image = generate_image(dataset)

    scale_factor = 0.1

    ph = get_image_value(image, point, "b0", scale_factor)
  
    return {"result": {"pH": ph}}

In [ ]:
@app.get("/land-surface-temperature-by-administrative-units-2")
async def get_land_surface_temperature_by_administrative_units_2(adm_0: str, adm_1: str, adm_2: str, start_date: datetime.date, end_date: datetime.date):
#def get_land_surface_temperature_by_administrative_units_2(adm_0: str, adm_1: str, adm_2: str, start_date: datetime.date, end_date: datetime.date):
    dataset = "MODIS/061/MOD11A1"
    provinces = ee.FeatureCollection("FAO/GAUL/2015/level2")
    geometry = provinces.filter(ee.Filter.eq('ADM0_NAME', adm_0)).filter(ee.Filter.eq('ADM1_NAME', adm_1)).filter(ee.Filter.eq('ADM2_NAME', adm_2)).geometry()

    image_collection = generate_collection(geometry, dataset, start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))

    result = get_mean(image_collection, geometry, "LST_Day_1km", 0.02, 1000)
    
    return {'result': result}

#get_land_surface_temperature_by_administrative_units_2("United States of America", "Hawaii", "Maui", "2020-01-01", "2020-05-01")

In [ ]:
@app.get("/soil-ph-by-administrative-units-2")
async def get_soil_ph_by_administrative_units_2(adm_0: str, adm_1: str, adm_2: str):
#def get_soil_ph_by_administrative_units_2(adm_0: str, adm_1: str, adm_2: str):

    dataset = "OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02"
    provinces = ee.FeatureCollection("FAO/GAUL/2015/level2")
    geometry = provinces.filter(ee.Filter.eq('ADM0_NAME', adm_0)).filter(ee.Filter.eq('ADM1_NAME', adm_1)).filter(ee.Filter.eq('ADM2_NAME', adm_2)).geometry()

    image = generate_image(dataset)

    scale_factor = 0.1

    ph = get_image_value(image, geometry, "b0", scale_factor)
  
    return {"result": {"pH": ph}}

#get_soil_ph_by_administrative_units_2("United States of America", "Hawaii", "Maui")

In [ ]:
@app.get("/worldpop")
async def get_worldpop(lat: float, lon: float, start_date: datetime.date, end_date: datetime.date):
    dataset = "WorldPop/GP/100m/pop"

    point = ee.Geometry.Point([lon, lat])

    image_collection = generate_collection(point, dataset, start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))

    result = None
    try:
        result = get_mean(image_collection, point, "population", 1, 100)
    except:
        result = None
    
    return {'result': result}

In [ ]:
@app.get("/elevation")
async def get_elevation(lat: float, lon: float):
    dataset = "NASA/NASADEM_HGT/001"

    point = ee.Geometry.Point([lon, lat])

    image = generate_image(dataset)

    scale_factor = 1

    elevation = None
    try:
        elevation = get_image_value(image, point, "elevation", scale_factor)
    except:
        elevation = None
    return {"result": {"elevation": elevation}}

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://9a89-35-238-195-40.ngrok-free.app


INFO:     Started server process [1245]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     210.61.66.205:0 - "GET / HTTP/1.1" 200 OK
INFO:     210.61.66.205:0 - "GET /worldpop?lat=35.701873319811384&lon=139.56197645704776&start_date=2020-01-01&end_date=2021-01-01 HTTP/1.1" 200 OK
INFO:     35.234.44.196:0 - "GET /worldpop?lat=41.773709&lon=140.726413&start_date=2020-01-01&end_date=2021-01-01 HTTP/1.1" 200 OK
INFO:     35.234.44.196:0 - "GET /elevation?lat=41.773709&lon=140.726413 HTTP/1.1" 200 OK
INFO:     35.234.44.196:0 - "GET /worldpop?lat=41.803557&lon=140.733539&start_date=2020-01-01&end_date=2021-01-01 HTTP/1.1" 200 OK
INFO:     35.234.44.196:0 - "GET /elevation?lat=41.803557&lon=140.733539 HTTP/1.1" 200 OK
INFO:     35.234.44.196:0 - "GET /worldpop?lat=41.846457&lon=140.722952&start_date=2020-01-01&end_date=2021-01-01 HTTP/1.1" 200 OK
INFO:     35.234.44.196:0 - "GET /elevation?lat=41.846457&lon=140.722952 HTTP/1.1" 200 OK
INFO:     35.234.44.196:0 - "GET /worldpop?lat=41.864641&lon=140.71358&start_date=2020-01-01&end_date=2021-01-01 HTTP/1.1" 200 OK
INFO:  

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1245]


## examples

* /land-surface-temperature?lat=52.72389418745157&lon=-92.03593750000002&start_date=2020-01-01&end_date=2020-05-01

* /soil-ph?lat=52.72389418745157&lon=-92.03593750000002

* /land-surface-temperature-by-administrative-units-2?adm_0=United%20States%20of%20America&adm_1=Hawaii&adm_2=Maui&start_date=2020-01-01&end_date=2020-05-01

* /soil-ph-by-administrative-units-2?adm_0=United%20States%20of%20America&adm_1=Hawaii&adm_2=Maui

* /worldpop?lat=35.701873319811384&lon=139.56197645704776&start_date=2020-01-01&end_date=2021-01-01

* /elevation?lat=52.72389418745157&lon=-92.03593750000002

In [ ]:
# kill ngrok instance
!killall -9 ngrok

ngrok: no process found


In [ ]:
### config middleware
"""
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)
"""


"\napp.add_middleware(\n    CORSMiddleware,\n    allow_origins=['*'],\n    allow_credentials=True,\n    allow_methods=['*'],\n    allow_headers=['*'],\n)\n"